In [82]:
import os
import json
import jsonschema

from pathlib import Path
from jsonschema import validate, ValidationError, Draft7Validator, RefResolver
from jsonschema.exceptions import RefResolutionError

import numpy as np

In [83]:
def get_schema(filename):
    """This function loads the given schema available"""
    with open(filename, 'r') as file:
        schema = json.load(file)
    return schema

In [89]:
def validate_json(schema, json_data):
    
    # https://stackoverflow.com/questions/25145160/json-schema-ref-does-not-work-for-relative-path
    schemas = (json.load(open(source)) for source in Path("schema").iterdir())
    schema_store = {schema["$id"]: schema for schema in schemas}
    
    resolver = RefResolver.from_schema(schema, store=schema_store)
    
    try:
        result = validate(instance=json_data, schema=schema, resolver=resolver)
        return True, None
    except ValidationError as err:
        return False, err.message

In [90]:
def validate_json2(schema, json_data):

    # https://stackoverflow.com/questions/25145160/json-schema-ref-does-not-work-for-relative-path
    schemas = (json.load(open(source)) for source in Path("schema").iterdir())
    schema_store = {schema["$id"]: schema for schema in schemas}    
    
    resolver = RefResolver.from_schema(schema, store=schema_store)
    validator = Draft7Validator(schema, resolver=resolver)

    try:
        errors = sorted(validator.iter_errors(json_data), key=lambda e: e.path)
    except RefResolutionError as e:
        print(e)

In [91]:
schema = get_schema(os.path.join('schema','effort_hours-report.json'))
schema

{'$id': 'effort_hours-report.json',
 '$schema': 'https://json-schema.org/draft/2020-12/schema',
 'description': 'An effort hours report for an individual Client',
 'additionalProperties': False,
 'type': 'object',
 'title': 'An effort report',
 'required': ['Client', 'Month-ending Date', 'Report Prepared Date'],
 'properties': {'Client': {'$ref': 'effort_hours-client.json'},
  'Month-ending Date': {'type': 'string',
   'format': 'date',
   'title': 'Hours calculated for the client for the month'},
  'Report Prepared Date': {'type': 'string',
   'format': 'date',
   'title': 'Hours calculated for the month'},
  'Notes': {'type': ['string', 'null'], 'title': 'Optional Client Notes'}}}

In [93]:
with open('onelab.json', 'r') as fp:
    onelab = json.load(fp)
validate_json(schema, onelab)

(True, None)